**TPM027A Cyber Risk Management (2022/23 Q1)** \
**Guest Lecture on *Cyber-physical security of water infrastructure*, Dr Riccardo Taormina**

# Detecting cyber-physical attacks on Smart Water Networks

## Introduction

### The case study

C-Town Public Utility (CPU) is the water distribution system operator of C-Town (see Figure 1). The network consists of 429 pipes, 388 junctions, 7 storage tanks, 11 pumps (distributed across 5 pumping stations), 1 actionable valve (V1), and a single reservoir (e.g., drinking water source). Water consumption is fairly regular throughout the year. 

C-Town is a Smart Water Network, where the physical assets are supervised by of 9 PLCs, which are located in proximity to pumps, storage tanks, and valves. Actuators are controlled based on high-low water levels in the respective controlling tank. Most of the PLCs controlling the pumps receive the information needed by the control logic from other PLCs—for instance, PLC1 controls Pumps PU1 and PU2 on the basis of Tank T1 water level, which is monitored by PLC2 (Table 1). 

All PLCs report sensed information to a central SCADA system. The SCADA historan contains a total of 43 variables, including: 7 water levels (identified by `L_\<tank_id>`), the status (on/off or open/closed) of pumps and valves (`S_\<actuator_id>`), the flow through the actuators (`F_\<actuator_id>`), and the pressure before and after the pumping stations (`P_\<junction_id>`).


| **PLC** | **Water level** |            **Status**            |             **Flow**             |                  **Pressure**                  |           **Actuators(controlling sensor)**          |
|:-------:|:---------------:|:--------------------------------:|:--------------------------------:|:----------------------------------------------:|:----------------------------------------------------:|
| PLC1    |  ---            | S_PU1, S_PU2, S_PU3              | F_PU1, F_PU2, F_PU3              | P_J280, P_J269                                 | PU1(L_T1), PU2(L_T1), PU3(=redundant pump)           |
| PLC2    | L_T1            |     ---                             |            ---                      |      ---                                          | ---                                                  |
| PLC3    | L_T2            | S_PU4, S_PU5, S_PU6, S_PU7, S_V1 | F_PU4, F_PU5, F_PU6, F_PU7, F_V1 | P_J300, P_J256, P_J289, P_J415, P_J302, P_J306 | V1(L_T2), PU4(L_T3), PU5(L_T3), PU6(L_T4), PU7(L_T4) |
| PLC4    | L_T3            |     ---                             |             ---                     |       ---                                         | ---                                                  |
| PLC5    | ---             | S_PU8, S_PU9, S_PU10, S_PU11     | F_PU8, F_PU9, F_PU10, F_PU11     | P_J307, P_J317, P_J14, P_J422                  | PU8(L_T5), PU9(L_T5), PU10(L_T7), PU11(L_T7)         |
| PLC6    | L_T4            |         ---                         |        ---                          |    ---                                            | ---                                                  |
| PLC7    | L_T5            |      ---                            |          ---                        |     ---                                           | ---                                                  |
| PLC8    | L_T6            |       ---                           |          ---                        |           ---                                     | ---                                                  |
| PLC9    | L_T7            |       ---                           |           ---                       |             ---                                   | ---                                                  |---

**Table 1.** Sensors/actuators (pumps and valves) monitored/controlled by PLCs. For each PLC, the corresponding controlling water level sensor provides the information needed to operate the actuators. A PLC-to-PLC connection is established whenever an actuator and the corresponding control sensor are connected to two different PLCs.

<p align="center">
    <img src="https://www.batadal.net/images/fig1.png" alt="The C-Town WDS" id="map" width="600px"/>
</p>     

**Figure 1.** The C-Town water distribution system, with highlighted PLCs, tanks and actuators. Only valve V1 is operated automatically.

Run the cell below to visualize in another web-page the sensed junctions before (blue) and after (white) the actuators, as well as the tanks (red). 

In [22]:
import wntr # The Water Network Tool for Resilience (WNTR) Python package simulate and analyze water distribution networks.

# load the C-Town computational model
inp_file = 'data/CTOWN.inp' 
wn = wntr.network.WaterNetworkModel(inp_file);

# visualize interactive map with all sensed junctions and tanks
p_nodes = [
    'J280','J269','J300','J256','J289','J415',
    'J302','J306','J307','J317','J14','J422',
    'T1','T2','T3','T4','T5','T6','T7']
v_nodes = [0,1,0,1,0,1,0,1,0,1,0,1,2,2,2,2,2,2,2]

wntr.graphics.plot_interactive_network(
    wn, 
    node_attribute=dict(zip(p_nodes, v_nodes)), node_cmap='Picnic', node_range=[0,2], add_colorbar = False, 
    filename='sensed_nodes.html', auto_open=True, figsize=(600,600), title='Sensed nodes || 0/1: pressure before/after actuator 2: tank');

C:\Users\rtaormina\Anaconda3\lib\site-packages\wntr\epanet\io.py:2131: UserWarning:

Not all curves were used in "data/CTOWN.inp"; added with type None, units conversion left to user



### Datasets

The are three datasets avaiable for the C-Town water distribution system, containing hourly SCADA historian observations for all recorded sensors during periods of normal operations as well as under cyber-physical attacks. These are:

1. `dataset_01.csv`: 365 days of normal observation data;
2. `dataset_02.csv`: 174 days of data containing 7 instances of cyber-physical attacks;
3. `dataset_03.csv`: 87 days of data containing 7 instances of cyber-physical attacks.

The attacks are various, performed on several devices and communication channels. Some of these attacks include deception attacks aimed at concealing altered signals by manipulating the data arriving to SCADA. One such example is reported in Figure 2 below; more sophisticated version include *replaying* previously recorded information (e.g., a bit like in hollywood movies where robbers play a tape to fool live CCTV recordings).

<p align="center">
    <img src="./images/concealment_attacks.PNG" alt="The C-Town WDS" id="map" width="800px"/>
</p>     

**Figure 2.** Attack \#3 from `dataset_02`. The attacker alters Tank T1 water level readings (solid line) sent by PLC2 to PLC1, which reads a constant low level (dotted line) and keeps Pumps PU1/PU2 on. This causes an overflow in Tank T1 (thick darkly shaded line). To conceal the action, the attacker alters the signal sent by PLC2 to the SCADA system (dashed line) by adding a time-varying offset (thin shaded line). The duration of the entire attack is highlighted by the thick lightly shaded line on the horizontal axis.

## Instructions

The exercise is divided in 4 parts:

1.  In the first part you will look at a dashboard of the SCADA historian for `dataset_01` to get an idea on the normal operating conditions of the smart water network. To enrich the analysis, we will also compute and plot the minimum, maximum, and median values for each sensor measured for each day of the week, for each hour of the week.


2. We will repeat what done before by looking at `dataset_02` for which all attacks are known and reported. Try to spot all the attacks using the information available, and comparing it against the weekly statistics (i.e., min, max, and median) computed for `dataset_01`. Does this comparison with expected normal operation conditions help the detection of the attacks?


3. Can you now detect the attacks in `dataset_03` without any information on them? Given the short time available, try to think which sensors are the most important based on your (basic) understanding of the water distribution systems and on the experience you gained in Part 2. Of course, you can work in teams!


4. Check your detection against the ground truth. Try to look at it only towards the end :) 

Enjoy being a cyber-physical security analysis!


## Part 1: load python modules and work on `dataset 1` (normal operations)

### Import python modules

In [23]:
# numpy stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# dashboard
from helper_functions import plot_dashboard

### Load data

In [24]:
# load data dataset
data_path = "./data/"
df01 = pd.read_csv(data_path + "dataset_01.csv", parse_dates = ['DATETIME'], dayfirst=True, index_col=0)

# remove (use example in commented lines if you want to customise your selected sensors)
sensor_cols = [col for col in df01.columns if (col not in ['ATT_FLAG'])]

# sensor_cols = [col for col in df.columns if (col not in ['DATETIME','ATT_FLAG']) & 
#                ('S_' not in col)  & ('P_' not in col) & ('F_' not in col)]

df01 = df01[sensor_cols]
df01.columns = pd.Index(sensor_cols,name='sensors')

### Compute statistics for each hour and day of the week

In [25]:
# quickly done using pandas (no need to understand what's done here)
df01_ = df01.copy()
df01_['dow'] = df01.reset_index()['DATETIME'].dt.dayofweek.values
df01_['hod'] = df01.reset_index()['DATETIME'].dt.hour.values
for stat in ['min','max','median']:
    df01_ = df01_.merge(
        df01_[sensor_cols+['dow','hod']].groupby(['dow','hod']).aggregate(stat).reset_index(), 
        how='left', on=['dow','hod'], suffixes=('', f'_{stat}'))
df01_.set_index(df01.index, inplace=True)

### Visualize dashboard for normal operations

In [26]:
app = plot_dashboard(df01_, sensor_cols, stats_to_show=['min','max'])
app.run_server(host="localhost",port=8051, debug=True)

Dash app running on http://localhost:8051/


## Part 2: identify known attacks in `dataset 2`

These are the 7 attacks in `dataset_02.csv`.

<p align="center">
    <img src="./images/attacks_dataset_02.PNG" alt="The C-Town WDS" id="map" width="800px"/>
</p>     


In [27]:
# load data dataset
df02 = pd.read_csv(data_path + "dataset_02.csv", parse_dates = ['DATETIME'], dayfirst=True, index_col=0)

df02_ = df02.copy()
df02_['dow'] = df02.reset_index()['DATETIME'].dt.dayofweek.values
df02_['hod'] = df02.reset_index()['DATETIME'].dt.hour.values
for stat in ['min','max','median']:
    df02_ = df02_.merge(
        df01_[sensor_cols+['dow','hod']].groupby(['dow','hod']).aggregate(stat).reset_index(), 
        how='left', on=['dow','hod'], suffixes=('', f'_{stat}'))

df02_.set_index(df02.index, inplace=True)

In [28]:
app = plot_dashboard(df02_, sensor_cols, stats_to_show=['min','max'], show_attacks=True)
app.run_server(host="localhost",port=8052, debug=True)

Dash app running on http://localhost:8052/


## Part 3: find unknown attacks in `dataset 3`

Use the dashboard again to find all 7 attacks in `dataset_03.csv`!

In [29]:
# load data dataset
df03 = pd.read_csv(data_path + "dataset_03.csv", parse_dates = ['DATETIME'], dayfirst=True, index_col=0)

In [30]:
df03_ = df03.copy()
df03_['dow'] = df03.reset_index()['DATETIME'].dt.dayofweek.values
df03_['hod'] = df03.reset_index()['DATETIME'].dt.hour.values
for stat in ['min','max','median']:
    df03_ = df03_.merge(
        df01_[sensor_cols+['dow','hod']].groupby(['dow','hod']).aggregate(stat).reset_index(), 
        how='left', on=['dow','hod'], suffixes=('', f'_{stat}'))

df03_.set_index(df03.index, inplace=True)

In [31]:
app = plot_dashboard(df03_, sensor_cols, stats_to_show=['min','max'], show_attacks=False)
app.run_server(host="localhost",port=8053, debug=True)

Dash app running on http://localhost:8053/


## Part 4: check against the ground truth

These are the 7 attacks in `dataset_03.csv`. How many did you find? Not an easy task indeed! That is why we need sophisticated intrusion detection methods. Run the last cell to check the dashboard 

<p align="center">
    <img src="./images/attacks_dataset_03.PNG" alt="The C-Town WDS" id="map" width="800px"/>
</p>     


In [32]:
app = plot_dashboard(df03_, sensor_cols, stats_to_show=['min','max'], show_attacks=True)
app.run_server(host="localhost",port=8054, debug=True)

Dash app running on http://localhost:8054/
